In [32]:
import requests

# Renfe Horarios de alta velocidad, larga distancia y media distancia
url = 'https://ssl.renfe.com/gtransit/Fichero_AV_LD/google_transit.zip'

# Download zip file
req = requests.get(url)

# Get file name from url
filename = url.split('/')[-1]

savepath = "datasets/"

# Writing the file to the local file system
with open(savepath+filename,'wb') as output_file:
    output_file.write(req.content)
print('Downloading Completed')

In [37]:
import shutil

# Unzip zip
shutil.unpack_archive(savepath+filename, savepath)

In [38]:
import pandas as pd

agency = pd.read_csv(savepath+"/agency.txt", sep=",")

agency.head()

,agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone
0,1071,RENFE OPERADORA,http://www.renfe.com,Europe/Madrid,ES,902320320


In [39]:
calendar_dates = pd.read_csv(savepath+"/calendar_dates.txt", sep=",")

calendar_dates.head()

,service_id,date,exception_type
0,2022-11-302022-12-10001901,20221204,2
1,2022-12-112023-03-12001901,20221211,2
2,2022-12-112023-03-12001901,20221218,2
3,2022-12-112023-03-12001901,20221225,2
4,2022-12-112023-03-12001901,20230101,2


In [21]:
# Parse date column
calendar_dates.date = calendar_dates.date.apply(lambda d : datetime.datetime.strptime(str(d), '%Y%m%d'))

In [22]:
calendar_dates.head()

,service_id,date,exception_type
0,2022-11-302022-12-10001901,2022-12-04,2
1,2022-12-112023-03-12001901,2022-12-11,2
2,2022-12-112023-03-12001901,2022-12-18,2
3,2022-12-112023-03-12001901,2022-12-25,2
4,2022-12-112023-03-12001901,2023-01-01,2
